Library	Purpose
numpy	Numerical operations
tensorflow / keras	Build deep learning model
Tokenizer	Convert words → numbers
pad_sequences	Make all sentences same length
Embedding	Convert word IDs → dense vectors
SimpleRNN	Core RNN layer
Dense	Final output layer

In [5]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, SimpleRNN, Dense

c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [ ]:


sentences = [
 "I love this product",
 "This movie made me smile",
 "Service was friendly and quick",
 "Today felt bright and happy",
 "This is the best day",
 "Absolutely fantastic experience",
 "I enjoyed every single moment",
 "Great job, well done",
 "The food tasted delicious",
 "Totally recommend to everyone",
 "Very satisfied with results",
 "This worked better than expected",
 "Amazing quality and value",
 "Such a pleasant surprise",
 "I feel positive about this",
 "I hate this product",
 "This movie bored me",
 "Service was rude and slow",
 "Today was cold and lonely",
 "This is the worst day",
 "Terrible experience overall",
 "I regret buying this",
 "Very disappointed with results",
 "The food tasted awful",
 "Do not recommend this",
 "It broke after one use",
 "Not worth the money",
 "Utterly frustrating and annoying",
 "I feel negative about this",
 "Such a waste of time",
]

labels= [1]*15 + [0]*15  # 1 for positive, 0 for negative  
#  THIS IS CREATED BECAUSE NEURAL NETWORKS WORK EFFIECIENTLY WITH NUMPY ARRAYS NOT PYTHON LISTS 
labels = np.array(labels) 


In [ ]:
# Maximum number of unique words the model will remember.
vocab_size = 2000
# IT CONVERTS WORDS INTO NUMBERS 
#  OOV TOKEN HANDLES UNKNOWN WORDS 
tok = Tokenizer(num_words=vocab_size,oov_token="")
# LEARNS A WORD DICTIONARY FROM ALL SENTENCES 
tok.fit_on_texts(sentences)
# IT CONVERTS SEQUENCE INTO NUMBERS SEQUENCES
# "I love this product" → [12, 45, 6, 98] 
seqs = tok.texts_to_sequences(sentences)
# FINDS LENGTH OF LONGEST SENTENCE 
# BASICALLY IT FINDS MAXM LENGTH THAT HOW MUCH LEN OF NUMBERS IS FORMING FROM WORDS TO NUMBERS SEQUENCE 
maxlen = max(len(s) for s in seqs)
# HERE WE ARE MAKING SAME LENGTH BY ADDING 0 AT THE END
X = pad_sequences(seqs,maxlen= maxlen, padding='post')
#  THIS IS CREATED BECAUSE NEURAL NETWORKS WORK EFFIECIENTLY WITH NUMPY ARRAYS NOT PYTHON LISTS 
y = np.array(labels)


In [8]:
# THIS IS THE NUMERICAL REPRESENTATION OF "I love this product" I.E FIRST SENTENCE
X[0]

array([ 3, 26,  2,  7,  0], dtype=int32)

In [ ]:
# EACH WORD HAS 16 DIM VECTOR
embed_dim = 16
# RNN HAS 8 MEMORY UNITS(NEURON)
rnn_units = 8

In [13]:
#  THIS IS INPUT LAYER IT RECIVES SEQUENCE OF WORDS NUMBERS AS INPUT
# LENGTH IS MAX LENGTH , DATA TYPE IS INT32
inp = Input(shape=(maxlen,), dtype='int32', name='input')
# IT CONVERTS WORD ID'S INTO MEANINGFUL VECTORS OF DIMENSION 16 
# MASK_ZERO=TRUE MEANS IGNORE ZERO PADDING IN SEQUENCE
x=Embedding(input_dim=vocab_size, output_dim = embed_dim, mask_zero=True, name='embed')(inp)
# SIMPLE RNN LAYER WITH 8 UNITS
# Reads words one by one

# Keeps memory of previous words

# Outputs only the final memory state
rnn= SimpleRNN(units=rnn_units, return_sequences=False, return_state=False, name='simple_rnn')
# Feeds embedded text into RNN
x_last = rnn(x)
# Converts RNN output → probability

# sigmoid → value between 0 and 1

# 1 = Positive, 0 = Negative
out=  Dense(1, activation='sigmoid', name='out')(x_last)
# Connects input → output

# Creates the final model
model = Model(inputs=inp, outputs=out)
# Adam → fast optimizer

# Binary crossentropy → yes/no classification

# Accuracy → performance metric
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# Displays model layers, shapes & parameters
model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input (InputLayer)  │ (None, 5)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embed (Embedding)   │ (None, 5, 16)     │     32,000 │ input[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_1         │ (None, 5)         │          0 │ input[0][0]       │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ simple_rnn          │ (None, 8)         │        200 │ embed[0][0],      │
│ (SimpleRNN)         │                   │            │ not_equal_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ out (Dense)         │ (None, 1)         │          9 │ simple_rnn[0][0]  │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 32,209 (125.82 KB)

 Trainable params: 32,209 (125.82 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:

# model.fit() trains the RNN by adjusting its weights over multiple epochs using batches of input data.
model.fit(X,y,epochs= 25,batch_size = 8,verbose = 1)
     

Epoch 1/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.5000 - loss: 0.6926
Epoch 2/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6667 - loss: 0.6769 
Epoch 3/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7667 - loss: 0.6629
Epoch 4/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8667 - loss: 0.6489
Epoch 5/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9000 - loss: 0.6350 
Epoch 6/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9000 - loss: 0.6195 
Epoch 7/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9333 - loss: 0.6038 
Epoch 8/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9333 - loss: 0.5872 
Epoch 9/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9667 - loss: 0.5700 
Epoch 10/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9667 - loss: 0.5511 
Epoch 11/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9667 - loss: 0.5308 
Epoch 12/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9667 - loss: 0.5101 
E

Task
Create an intermediate Keras model to visualize the outputs of the embedding layer and the hidden states of the SimpleRNN layer from the existing model. Then, use this intermediate model to predict on the input data and visualize the outputs.

Create intermediate model
Subtask:
Build a new Keras Model that takes the same input as the original model but outputs the embedding layer and the hidden states of the SimpleRNN layer.

Reasoning: Define an intermediate Keras model with the same input as the original model and outputs from the embedding and simple RNN layers.

In [15]:

intermediate_model = Model(inputs=model.inputs, outputs=[model.get_layer('embed').output, model.get_layer('simple_rnn').output])
     

In [16]:

from tensorflow.keras.layers import SimpleRNN as SRNN
seq_inp = Input(shape=(maxlen,), dtype='int32')
seq_emb = model.get_layer('embed')(seq_inp)  # reuse trained embedding

# Create RNN with return_sequences=True
rnn_seq = SRNN(units=rnn_units, return_sequences=True, name='rnn_seq')

# DO NOT CALL build() manually
seq_hidden = rnn_seq(seq_emb)  # builds automatically

# Copy trained RNN weights
try:
    trained_weights = model.get_layer('simple_rnn').get_weights()
    rnn_seq.set_weights(trained_weights)
    print("Copied RNN weights into sequence-inspection RNN.")
except Exception as e:
    print("Could not copy weights automatically:", e)

inspect_model = Model(inputs=seq_inp, outputs=seq_hidden)

# Inspect
idx = 0
example_seq = X[idx:idx+1]  # shape (1, maxlen)
hidden_seq = inspect_model.predict(example_seq)

print("Sentence:", sentences[idx])
print("Token ids:", example_seq)
print("Hidden states per timestep shape:", hidden_seq.shape)
print("Hidden states (timesteps x units):")
print(np.round(hidden_seq[0], 3))

Copied RNN weights into sequence-inspection RNN.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 410ms/step
Sentence: I love this product
Token ids: [[ 3 26  2  7  0]]
Hidden states per timestep shape: (1, 5, 8)
Hidden states (timesteps x units):
[[ 0.038  0.01  -0.039 -0.001  0.003 -0.034  0.018  0.06 ]
 [ 0.262 -0.133  0.033  0.006  0.139 -0.098 -0.142  0.074]
 [-0.03  -0.112  0.292  0.284 -0.183  0.139  0.117  0.042]
 [ 0.115  0.014  0.073 -0.126  0.249 -0.396 -0.417  0.186]
 [ 0.115  0.014  0.073 -0.126  0.249 -0.396 -0.417  0.186]]


Built an inspection model that reveals the hidden state of the trained RNN at each word in the sentence.

IN THIS CODE WE BASCIALLY BUILT AN INSPECTION MODEL THAT REVEALS THE HIDDEN STATE OF THE TRAINED RNN AT EACH WORD IN THE SENTENCE.

PICK FIRST SENTENCE AND PREDICT HIDDEN STATES SUCH AS TOKEN ID AND HIDDEN STATES


Dimension	Meaning
1	One sentence
5	5 time-steps (5 tokens)
8	8 RNN memory units

Token ids: [[ 3 26  2  7  0]]
IN TOKEN IDS 0 MEANS WE ADDED PADDING


🔹 Hidden states printed
[[-0.016 -0.011 -0.013  0.053  0.073 -0.071  0.018 -0.019]   ← after "I"
 [ 0.045 -0.046 -0.210 -0.094 -0.181 -0.020 -0.137  0.099]   ← after "love"
 [-0.010 -0.243 -0.152 -0.143  0.133  0.031  0.351 -0.113]   ← after "this"
 [-0.336 -0.099 -0.260  0.179 -0.012 -0.316 -0.127 -0.114]   ← after "product"
 [-0.336 -0.099 -0.260  0.179 -0.012 -0.316 -0.127 -0.114]   ← padding (0)